In [ ]:
!pip install pandas numpy statsmodels matplotlib


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller


Cargando dataset y uniendo columnas 'Dia', 'Mes', 'Año' en Date

In [ ]:
df_ap = pd.read_csv('data/ML_data_csv/anexo_circular_7_atencion_primaria.csv')
print(df_ap.columns)

In [ ]:
month_mapping = { 'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12 } 
# Replace month names with numbers 
df_ap['Mes'] = df_ap['Mes'].map(month_mapping)
# Ensure Dia, Mes, Año columns are of integer type 
df_ap['Dia'] = df_ap['Dia'].astype(int) 
df_ap['Mes'] = df_ap['Mes'].astype(int) 
df_ap['Año'] = df_ap['Año'].astype(int)
print(df_ap[['Dia', 'Mes', 'Año']].isnull().sum())

In [5]:

# Combine Dia, Mes, Año into a single datetime column 
df_ap['Date'] = pd.to_datetime(df_ap[['Dia', 'Mes', 'Año']].astype(str).agg('-'.join, axis=1), format='%d-%m-%Y')

df_ap.set_index('Date', inplace=True)



In [ ]:
df_ap.head(10)

In [7]:
df_ap.drop(columns=['Dia', 'Mes', 'Año'], inplace=True) 

 

Guardando nombres de las provincias que aparecen en el dataset para que cada provincia sea una llave en el diccionario province_data_dict y su valor sean los datos correspondientes a esa provincia. Luego se organizan por fecha los datos de cada provincia y se revisa si hay duplicados o valores NAN.

In [ ]:
# Iterate over each province 
provinces = df_ap['Provincias'].unique()
print(provinces)
provinces = provinces[:-1] # to delete Total
print(provinces)

In [ ]:
province_data_dict = {}

# Iterate over each province and store the data in the dictionary
for province in provinces:
    province_data = df_ap[df_ap['Provincias'] == province]
    province_data.drop(columns=['Provincias'], inplace=True) 
    province_data_dict[province] = province_data
    province_data_dict[province] = province_data_dict[province].sort_index()
    # Check for duplicate dates 
    duplicate_dates = province_data_dict[province].index.duplicated(keep=False) 
    if duplicate_dates.any(): 
        print(f"Duplicate dates found for {province}:") 
        print(province_data_dict[province][duplicate_dates])      
    
    # Check for NaN values
    nan_values = province_data.isna().sum()
    if nan_values.any():
        print(f"NaN values for {province}:")
        print(nan_values)
    # Check for string data types 
    for column in province_data.columns: 
        if province_data[column].dtype == 'object': 
            print(f"Column '{column}' in province '{province}' contains string data.")
    
    
    
    

En Guantánamo hay un valor Nan value en Hogar_A_Fallecidos. Por ahora se va a ignorar esta columna para las predicciones de esta provincia. También se aprecia que la fecha 2023-09-03 (3 de sept del 2023) aparece repetida(con dos filas) en cada provincia. Al revisar el dataset se comprobó que en la entrada correspondiente al 4 de septiembre escribieron 3 de septiembre por error; por tanto se eliminará la repetición de esta cambiándola por 4 de septiembre.

In [10]:
province_data_dict['Guantánamo'].drop(columns=['Hogar_A_Fallecidos'], inplace=True) 
nan_values = province_data_dict['Guantánamo'].isna().sum()
#Checking for any remaining nan values
if nan_values.any():
    print(f"NaN values for 'Guantánamo':")
    print(nan_values)



In [ ]:
for province in provinces:
    # Check for duplicate dates 
    duplicates = province_data_dict[province].index.duplicated(keep='first') 
    print(province)
    print()
    print(f'{province_data_dict[province][duplicates]} in {province}')
    duplicate_dates = province_data_dict[province][duplicates].index 
    # Modify the second instance of each duplicate date 
    for date in duplicate_dates: 
        # Find all instances of the duplicate date
        all_instances_idx = province_data_dict[province].index.get_loc(date)
        # Ensure all_instances_idx is a list
        if isinstance(all_instances_idx, slice):
            all_instances_idx = list(range(all_instances_idx.start, all_instances_idx.stop))
        elif isinstance(all_instances_idx, int):
            all_instances_idx = [all_instances_idx]
        # Modify the date of the second instance
        if len(all_instances_idx) > 1:
            second_instance_idx = all_instances_idx[1]
            province_data_dict[province].index.values[second_instance_idx] = province_data_dict[province].index[second_instance_idx] + pd.Timedelta(days=1)
    duplicates = province_data_dict[province].index.duplicated(keep='first') 
    if duplicates.any():
        print()
        print("After dealing with duplicates")
        print(f'{province_data_dict[province][duplicates]} still in {province}')


In [ ]:
print(province_data_dict['Pinar del Rio'].loc['2023-09-03':'2023-09-04'])

## Empezando a predecir usando ARIMA.

Por cada provincia se estimarán los parámetros p y q con ACF y PACF y revisaremos si coincide con los que da autoArima. También comprobaremos si los datos son estacionarios, lo cual es un requerimiento para ARIMA.
Columnas a predecir:
Hogar_I_Total	Hogar_I_<19aÃ±os	Hogar_A_Recuperados	Hogar_A_Remitidos_ingreso_institucional	Hogar_A_Fallecidos	Hogar_A_Total	Hogar_PI_Total	Hogar_PI_<19aÃ±os	Policlinico_I_Total	Policlinico_I_<19aÃ±os	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19aÃ±os	TotalAPS_I_Total	TotalAPS_I_<19aÃ±os	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19aÃ±os	PIDA_hogar	PIDA_hogar_<19aÃ±os	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19aÃ±os	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


## Hogar_I_Total

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ## Hogar_I_<19años	Hogar_A_Recuperados	Hogar_A_Remitidos_ingreso_institucional	Hogar_A_Fallecidos	Hogar_A_Total	Hogar_PI_Total	Hogar_PI_<19años	Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Hogar_I_Total']
    print("Hogar_I_Total")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()



#     # Fit ARIMA model (example with p=1, d=1, q=1)
#     model = ARIMA(series, order=(1, 1, 1))
#     model_fit = model.fit()
#     print(model_fit.summary())
    
#     # Forecast
#     forecast = model_fit.forecast(steps=10)
#     predictions[province] = forecast

# # Print predictions
# for province, forecast in predictions.items():
#     print(f'Predictions for {province}:')
#     print(forecast)


This should be used to determine the parameters for ARIMA. 
ACF Plot
The ACF plot shows the correlation between the time series and its lagged values. It helps identify the Moving Average (MA) component of the ARIMA model.

MA(q) Component: Look for the lag where the ACF plot cuts off (i.e., drops to zero or near zero). The lag at which this happens indicates the order of the MA component (q).

PACF Plot
The PACF plot shows the correlation between the time series and its lagged values, after removing the effects of intermediate lags. It helps identify the AutoRegressive (AR) component of the ARIMA model.

AR(p) Component: Look for the lag where the PACF plot cuts off. The lag at which this happens indicates the order of the AR component (p).

Steps to Determine ARIMA Parameters
Plot ACF and PACF: Generate the ACF and PACF plots for your time series data.

Identify AR(p) and MA(q): Use the ACF and PACF plots to determine the orders of the AR and MA components.

Determine Differencing (d): Use the Augmented Dickey-Fuller (ADF) test to check for stationarity. If the series is not stationary, apply differencing until it becomes stationary. The number of times you difference the series is the order of differencing (d).

## Predicciones

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_I_Total. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_I_Total. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Calculate evaluation metrics
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((test - predictions) / test)) * 100
nmse = mse / np.var(test)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')
print(f'NMSE: {nmse}')

# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
plt.plot(test.index, predictions, 'x', label='Predicted', color='red') # Use 'x' for markers 
plt.legend()
plt.show()
print(model_fit.summary()) 



## Hogar_I_<19años

In [ ]:
province_data_dict["Artemisa"].head()
province_data_dict["Artemisa"]["Hogar_I_<19años"]

In [ ]:
from arima_utils import auto_arima_train

predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "Hogar_I_<19años")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_I_<19años. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_I_<19años. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Hogar_A_Recuperados

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Hogar_A_Remitidos_ingreso_institucional	Hogar_A_Fallecidos	Hogar_A_Total	Hogar_PI_Total	Hogar_PI_<19años	Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Hogar_A_Recuperados']
    print("Hogar_A_Recuperados")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()


## Predictions


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_A_Recuperados. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_A_Recuperados. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Hogar_A_Remitidos_ingreso_institucional

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##		Hogar_A_Fallecidos	Hogar_A_Total	Hogar_PI_Total	Hogar_PI_<19años	Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Hogar_A_Remitidos_ingreso_institucional']
    print("Hogar_A_Remitidos_ingreso_institucional")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()


## predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_A_Remitidos_ingreso_institucional. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_A_Remitidos_ingreso_institucional. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Hogar_A_Fallecidos

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##Hogar_A_Total	Hogar_PI_Total	Hogar_PI_<19años	Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    try:
        series = province_data["Hogar_A_Fallecidos"]
    except:
        continue
    print("Hogar_A_Fallecidos")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()



## Prediction

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in models.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_A_Fallecidos. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_A_Fallecidos. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Hogar_A_Total

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Hogar_PI_Total	Hogar_PI_<19años	Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Hogar_A_Total']
    print("Hogar_A_Total")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()




## Predictions

In [ ]:


from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_A_Total. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_A_Total. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Hogar_PI_Total	

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Hogar_PI_<19años	Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Hogar_PI_Total']
    print("Hogar_PI_Total	")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()
 

## Predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_PI_Total	. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_PI_Total	. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Hogar_PI_<19aÃ±os

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##		Policlinico_I_Total	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Hogar_PI_<19años']
    print("Hogar_PI_<19años")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()


## Predictions

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Hogar_PI_<19años. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Hogar_PI_<19años. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_I_Total

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Policlinico_I_<19años	Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Policlinico_I_Total']
    print("Policlinico_I_Total")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()


## Predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Policlinico_I_Total. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Policlinico_I_Total. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_I_<19aÃ±os

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##		Policlinico_A_Recuperados	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Policlinico_I_<19años']
    print("Policlinico_I_<19años")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()



## Predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Policlinico_I_<19años. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Policlinico_I_<19años. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_A_Recuperados

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Policlinico_A_Remitidos_ingreso_hospitalario	Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Policlinico_A_Recuperados']
    print("Policlinico_A_Recuperados")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()



## Predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Policlinico_A_Recuperados. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Policlinico_A_Recuperados. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_A_Remitidos_ingreso_hospitalario

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##		Policlinico_A_Fallecidos	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Policlinico_A_Remitidos_ingreso_hospitalario']
    print("Policlinico_A_Remitidos_ingreso_hospitalario")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()



## Predictions

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Policlinico_A_Remitidos_ingreso_hospitalario. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Policlinico_A_Remitidos_ingreso_hospitalario. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_A_Fallecidos

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Policlinico_A_Total	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Policlinico_A_Fallecidos']
    print("Policlinico_A_Fallecidos")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()


## Predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Policlinico_A_Fallecidos. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Policlinico_A_Fallecidos. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_A_Total

In [ ]:
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
 ##	Policlinico_PI_Total	Policlinico_PI_<19años	TotalAPS_I_Total	TotalAPS_I_<19años	TotalAPS_A_Recuperados	TotalAPS_A_Remitidos	TotalAPS_A_Fallecidos	TotalAPS_A_Total	TotalAPS_PI_Total	TotalAPS_PI_<19años	PIDA_hogar	PIDA_hogar_<19años	PIDA_hospitalizaciÃ³n	PIDA_hospitalizaciÃ³n_<19años	ValidaciÃ³n_PIactual	ADAH_Ingresos	ADAH_AltasRecuperados	ADAH_Fallecidos	DH_Ingresos	DH_AltasRecuperados	DH_Fallecidos	AAH_Ingresos	AAH_AltasRecuperados	AAH_Fallecidos	ADAP_Ingresos	ADAP_AltasRecuperados	ADAP_Fallecidos	DP_Ingresos	DP_AltasRecuperados	DP_Fallecidos	AAP_Ingresos	AAP_AltasRecuperados	AAP_Fallecidos	AATotalAPS_Ingresos	AATotalAPS_AltasRecuperados	AATotalAPS_Fallecidos


predictions = {}
models = {}
test_sets = {}
d_from_diff = {}
train_sets = {}

# Iterate over each province
for province in province_data_dict.keys():
    # Filter data for the current province
    province_data = province_data_dict[province]
    
    # Extract the series to be predicted
    series = province_data['Policlinico_A_Total']
    print("Policlinico_A_Total")
    print(province)
    print()
    
    # Split the data into training and test sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    test_sets[province] = test
    train_sets[province] = train

    # Use auto_arima to find the best parameters on the training set
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    models[province] = model
    
    if train.nunique() == 1: 
        print(f"The series for province '{province}' is constant.\n") 
        d_from_diff[province] = 0
        continue
    
    # Check for stationarity and apply differencing if needed
    d = 0
    result = adfuller(train)
    while result[1] > 0.05:
        train = train.diff().dropna()
        d += 1
        result = adfuller(train)
    d_from_diff[province] = d
    print(f"Best parameters for province '{province}': (d={d} from differencing, d={model.order[1]} d from AutoArima)\n")
    # Create subplots for ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(train, ax=axes[0])
    plot_pacf(train, ax=axes[1])
    axes[0].set_title(f'ACF for {province} (MA={model.order[2]})')
    axes[1].set_title(f'PACF for {province} (AR={model.order[0]})')
    plt.tight_layout()
    plt.show()


## Predictions

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for province in province_data_dict.keys():
    auto_model = models[province]
    test = test_sets[province]
    
    auto_predictions = auto_model.predict(n_periods=len(test))
    d = d_from_diff[province]
    # Fit ARIMA model on the training set
    model = ARIMA(train_sets[province], order = (auto_model.order[0], d, auto_model.order[2]))
    model_fit = model.fit()
    # Make predictions on the test set
    predictions = model_fit.forecast(steps=len(test))
    print(f' AutoArima Predictions for {province}: Policlinico_A_Total. d = {auto_model.order[1]}') 
    print(auto_predictions)
    print()
    print(f'Arima Predictions for {province}: Policlinico_A_Total. d = {d}') 
    print(predictions)


    # Calculate evaluation metrics
    auto_mae = mean_absolute_error(test, auto_predictions)
    auto_mse = mean_squared_error(test, auto_predictions)
    auto_rmse = np.sqrt(auto_mse)
    auto_mape = np.mean(np.abs((test - auto_predictions) / test)) * 100
    auto_nmse = auto_mse / np.var(test)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, predictions)
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - predictions) / test)) * 100
    nmse = mse / np.var(test)

    print(f'MAE for autoArima: {auto_mae}')
    print(f'MSE for autoArima: {auto_mse}')
    print(f'RMSE for autoArima: {auto_rmse}')
    print(f'MAPE for autoArima: {auto_mape}%')
    print(f'NMSEfor autoArima: {auto_nmse}')

    print()
    print(f'MAE for ARIMA: {mae}')
    print(f'MSE for ARIMA: {mse}')
    print(f'RMSE for ARIMA: {rmse}')
    print(f'MAPE for ARIMA: {mape}%')
    print(f'NMSE for ARIMA: {nmse}')

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, auto_predictions, 'x', label='AutoArima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test.index, test, 'o', label='Actual') # Use 'o' for markers 
    plt.plot(test.index, predictions, 'x', label='Arima Predicted', color='red') # Use 'x' for markers 
    plt.legend()
    plt.show()
    print(model_fit.summary()) 

## Policlinico_PI_Total

In [ ]:
from arima_utils import auto_arima_train, evaluate_models

predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "Policlinico_PI_Total")

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "Policlinico_PI_Total")

## Policlinico_PI_<19aÃ±os

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "Policlinico_PI_<19años")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "Policlinico_PI_<19años")

##	TotalAPS_I_Total

In [ ]:


predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_I_Total")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_I_Total")

## TotalAPS_I_<19aÃ±os

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_I_<19años")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_I_<19años")

## TotalAPS_A_Recuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_A_Recuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_A_Recuperados")

## TotalAPS_A_Remitidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_A_Remitidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_A_Remitidos")


## TotalAPS_A_Fallecidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_A_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_A_Fallecidos")


## TotalAPS_A_Total	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_A_Total")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_A_Total")


## TotalAPS_PI_Total	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_PI_Total")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_PI_Total")


## TotalAPS_PI_<19años	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "TotalAPS_PI_<19años")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "TotalAPS_PI_<19años")


## PIDA_hogar	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "PIDA_hogar")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "PIDA_hogar")


## PIDA_hogar_<19aÃ±os	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "PIDA_hogar_<19años")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "PIDA_hogar_<19años")


## PIDA_hospitalizaciÃ³n

In [68]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "PIDA_hospitalizaciÃ³n")

## Predictions

In [69]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "PIDA_hospitalizaciÃ³n")


## PIDA_hospitalizaciÃ³n_<19aÃ±os	

In [70]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "PIDA_hospitalizaciÃ³n_<19años")

## Predictions

In [71]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "PIDA_hospitalizaciÃ³n_<19años")


## ValidaciÃ³n_PIactual	

In [72]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ValidaciÃ³n_PIactual")

## Predictions

In [73]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ValidaciÃ³n_PIactual")


## ADAH_Ingresos

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAH_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAH_Ingresos")


## ADAH_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAH_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAH_Ingresos")


## ADAH_AltasRecuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAH_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAH_AltasRecuperados")


## ADAH_Fallecidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAH_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAH_Fallecidos")


## DH_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "DH_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "DH_Ingresos")


## DH_AltasRecuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "DH_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "DH_AltasRecuperados")


## DH_Fallecidos

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "DH_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "DH_Fallecidos")


## AAH_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AAH_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AAH_Ingresos")


## AAH_AltasRecuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AAH_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AAH_AltasRecuperados")


## AAH_Fallecidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AAH_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AAH_Fallecidos")


## ADAP_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAP_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAP_Ingresos")


## ADAP_AltasRecuperados

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAP_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAP_AltasRecuperados")


## ADAP_Fallecidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "ADAP_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "ADAP_Fallecidos")


## DP_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "DP_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "DP_Ingresos")


## DP_AltasRecuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "DP_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "DP_AltasRecuperados")


## DP_Fallecidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "DP_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "DP_Fallecidos")


## AAP_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AAP_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AAP_Ingresos")


## AAP_AltasRecuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AAP_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AAP_AltasRecuperados")


## AAP_Fallecidos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AAP_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AAP_Fallecidos")


## AATotalAPS_Ingresos	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AATotalAPS_Ingresos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AATotalAPS_Ingresos")


## AATotalAPS_AltasRecuperados	

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AATotalAPS_AltasRecuperados")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AATotalAPS_AltasRecuperados")


## AATotalAPS_Fallecidos

In [ ]:
predictions, models, test_sets, d_from_diff, train_sets = auto_arima_train(province_data_dict, "AATotalAPS_Fallecidos")

## Predictions

In [ ]:
evaluate_models(models, test_sets, d_from_diff, train_sets, "AATotalAPS_Fallecidos")
